In [25]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv("../data/bebidas-no-alcoholicas-pivot.csv").query(
    "Mercado == 'Mexico'"
)

df

,Mercado,Categoria,Presentacion,Fecha,Sabor,Dist. Num.,Precio EQ2 Promedio,Precio Unds Promedio,Vtas Unds,Vtas Valor,Vtas Volumen
40582,Mexico,AGUA NATURAL,FAMILIAR,2021-12-01,NOT APPLICABLE,49.640361,26.455,26.455,8.678932e+06,2.295996e+08,8.678932e+06
40583,Mexico,AGUA NATURAL,FAMILIAR,2022-01-01,NOT APPLICABLE,49.483236,26.847,26.847,9.094049e+06,2.441441e+08,9.094049e+06
40584,Mexico,AGUA NATURAL,FAMILIAR,2022-02-01,NOT APPLICABLE,48.184131,27.182,27.182,7.895758e+06,2.146197e+08,7.895758e+06
40585,Mexico,AGUA NATURAL,FAMILIAR,2022-03-01,NOT APPLICABLE,50.726610,27.328,27.328,9.578185e+06,2.617485e+08,9.578185e+06
40586,Mexico,AGUA NATURAL,FAMILIAR,2022-04-01,NOT APPLICABLE,51.805138,27.489,27.489,1.078616e+07,2.965004e+08,1.078616e+07
...,...,...,...,...,...,...,...,...,...,...,...
49405,Mexico,TE LISTO PARA BEBER,PACK,2024-08-01,KOMBUCHA/OTR. SABORES,0.000084,150.000,150.000,1.000000e+00,1.500000e+02,1.000000e+00
49406,Mexico,TE LISTO PARA BEBER,PACK,2024-10-01,NEGRO/LIMON,0.000084,2.592,2.592,1.000000e+00,2.592000e+00,1.000000e+00
49407,Mexico,TE LISTO PARA BEBER,PACK,2024-10-01,VERDE/LIMON,0.000084,2.592,2.592,1.000000e+00,2.592000e+00,1.000000e+00
49408,Mexico,TE LISTO PARA BEBER,PACK,2024-12-01,OTR. TIPOS/OTR. SABORES,0.000084,24.440,24.440,1.000000e+00,2.444000e+01,1.000000e+00


## ¿Cómo ha sido la evolución entre refrescos de colas y sabores?

In [20]:
cola = df.query("Sabor == 'COLA'").groupby("Fecha").sum().reset_index()
other = df.query("Sabor != 'COLA'").groupby("Fecha").sum().reset_index()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="Cola",
        mode="markers+lines",
        x=cola["Fecha"],
        y=cola["Vtas Valor"],
        marker_symbol="star",
    )
)

fig.add_trace(
    go.Scatter(
        name="Otros sabores",
        mode="markers+lines",
        x=other["Fecha"],
        y=other["Vtas Valor"],
        marker_symbol="star",
    )
)

title = "Ventas de Bebidas No Alcohólicas\n<br><i>2022 a 2024</i>"
fig.update_layout(
    title=title,
    title_x=0.5,
    yaxis_title="Ventas ($)",
    barcornerradius=5,
    paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
    plot_bgcolor="rgba(0,0,0,0)",
)

fig.add_vrect(
    x0="2022-01-01",
    x1="2022-12-31",
    fillcolor="blue",
    opacity=0.1,
    layer="below",
    line_width=0,
)

fig.add_vrect(
    x0="2023-01-01",
    x1="2023-12-31",
    fillcolor="gray",
    opacity=0.1,
    layer="below",
    line_width=0,
)

fig.add_vrect(
    x0="2024-01-01",
    x1="2024-12-31",
    fillcolor="blue",
    opacity=0.1,
    layer="below",
    line_width=0,
)


fig.write_image(
    "../img/01-questions/01-ventas-mes.png",
    scale=3,
    width=1000,
    height=600,
)
fig.show()

## ¿Cuál es más sensible ante incrementos en precio?

In [34]:
cola = (
    df.query("Sabor == 'COLA'")
    .groupby("Fecha")
    .aggregate(
        {
            "Precio Unds Promedio": "mean",
            "Vtas Valor": "sum",
        }
    )
    .reset_index()
)

cola["Last Vtas Valor"] = cola["Vtas Valor"].shift(1)
cola["Last Precio Unds Promedio"] = cola["Precio Unds Promedio"].shift(1)

cola["Vtas Valor Growth"] = cola["Vtas Valor"] - cola["Last Vtas Valor"]
cola["Precio Unds Promedio Change"] = (
    cola["Precio Unds Promedio"] - cola["Last Precio Unds Promedio"]
).abs()

cola["Vtas Valor Growth per unit price change"] = (
    cola["Vtas Valor Growth"] / cola["Precio Unds Promedio Change"]
)

other = (
    df.query("Sabor != 'COLA'")
    .groupby("Fecha")
    .aggregate(
        {
            "Precio Unds Promedio": "mean",
            "Vtas Valor": "sum",
        }
    )
    .reset_index()
)

other["Last Vtas Valor"] = other["Vtas Valor"].shift(1)
other["Last Precio Unds Promedio"] = other["Precio Unds Promedio"].shift(1)

other["Vtas Valor Growth"] = other["Vtas Valor"] - other["Last Vtas Valor"]
other["Precio Unds Promedio Change"] = (
    other["Precio Unds Promedio"] - other["Last Precio Unds Promedio"]
).abs()

other["Vtas Valor Growth per unit price change"] = (
    other["Vtas Valor Growth"] / other["Precio Unds Promedio Change"]
)

cola

,Fecha,Precio Unds Promedio,Vtas Valor,Last Vtas Valor,Last Precio Unds Promedio,Vtas Valor Growth,Precio Unds Promedio Change,Vtas Valor Growth per unit price change
0,2021-12-01,48.03925,1.449692e+10,NaN,NaN,NaN,NaN,NaN
1,2022-01-01,49.69000,1.403562e+10,1.449692e+10,48.03925,-4.613012e+08,1.65075,-2.794494e+08
2,2022-02-01,52.74075,1.272066e+10,1.403562e+10,49.69000,-1.314961e+09,3.05075,-4.310287e+08
3,2022-03-01,49.34875,1.477094e+10,1.272066e+10,52.74075,2.050288e+09,3.39200,6.044480e+08
4,2022-04-01,49.63225,1.534501e+10,1.477094e+10,49.34875,5.740645e+08,0.28350,2.024919e+09
5,2022-05-01,51.50800,1.666867e+10,1.534501e+10,49.63225,1.323666e+09,1.87575,7.056731e+08
6,2022-06-01,49.96125,1.647689e+10,1.666867e+10,51.50800,-1.917884e+08,1.54675,-1.239944e+08
7,2022-07-01,50.24625,1.716693e+10,1.647689e+10,49.96125,6.900440e+08,0.28500,2.421207e+09
8,2022-08-01,51.51400,1.709344e+10,1.716693e+10,50.24625,-7.348593e+07,1.26775,-5.796563e+07
9,2022-09-01,51.67575,1.636239e+10,1.709344e+10,51.51400,-7.310520e+08,0.16175,-4.519641e+09


In [39]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="Cola",
        mode="markers+lines",
        x=cola["Fecha"],
        y=cola["Vtas Valor Growth per unit price change"],
        marker_symbol="star",
    )
)

fig.add_trace(
    go.Scatter(
        name="Otros sabores",
        mode="markers+lines",
        x=other["Fecha"],
        y=other["Vtas Valor Growth per unit price change"],
        marker_symbol="star",
    )
)

title = "Crecimiento de Ventas por Cambio de Precios\n<br><i>2022 a 2024</i>"
fig.update_layout(
    title=title,
    title_x=0.5,
    yaxis_title="Ventas ($) / Cambio de precio",
    barcornerradius=5,
    paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
    plot_bgcolor="rgba(0,0,0,0)",
)

fig.add_vrect(
    x0="2022-01-01",
    x1="2022-12-31",
    fillcolor="blue",
    opacity=0.1,
    layer="below",
    line_width=0,
)

fig.add_vrect(
    x0="2023-01-01",
    x1="2023-12-31",
    fillcolor="gray",
    opacity=0.1,
    layer="below",
    line_width=0,
)

fig.add_vrect(
    x0="2024-01-01",
    x1="2024-12-31",
    fillcolor="blue",
    opacity=0.1,
    layer="below",
    line_width=0,
)


fig.write_image(
    "../img/01-questions/02-sensibilidad-precios.png",
    scale=3,
    width=1000,
    height=600,
)

fig.show()

In [ ]:
RELEVANT_MONTHS = [4, 5, 6, 7, 8]

cola = (
    df.query("Sabor == 'COLA'")
    .astype({"Fecha": "datetime64[ns]"})
    .query("Fecha.dt.month.isin(@RELEVANT_MONTHS)")
    .groupby("Fecha")
    .aggregate(
        {
            "Vtas Unds": "sum",
            "Vtas Valor": "sum",
        }
    )
    .reset_index()
)

cola


,Fecha,Vtas Unds,Vtas Valor
0,2022-04-01,8.208119e+08,1.534501e+10
1,2022-05-01,8.872521e+08,1.666867e+10
2,2022-06-01,8.682882e+08,1.647689e+10
3,2022-07-01,8.927093e+08,1.716693e+10
4,2022-08-01,8.772189e+08,1.709344e+10
5,2023-04-01,8.244527e+08,1.710524e+10
6,2023-05-01,8.634423e+08,1.802651e+10
7,2023-06-01,8.767476e+08,1.835174e+10
8,2023-07-01,9.286125e+08,1.961986e+10
9,2023-08-01,8.949635e+08,1.902245e+10


In [ ]:
# cola = df.query("Sabor == 'COLA'").groupby("Fecha")["Vtas Valor"].sum().reset_index()
# cola["Last Vtas Valor"] = cola["Vtas Valor"].shift(1)
# cola["Diff Vtas Valor"] = cola["Vtas Valor"] - cola["Last Vtas Valor"]

# other = df.query("Sabor != 'COLA'").groupby("Fecha")["Vtas Valor"].sum().reset_index()
# other["Last Vtas Valor"] = other["Vtas Valor"].shift(1)
# other["Diff Vtas Valor"] = other["Vtas Valor"] - other["Last Vtas Valor"]

# fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         name="Cola",
#         mode="markers+lines",
#         x=cola["Fecha"],
#         y=cola["Diff Vtas Valor"],
#         marker_symbol="star",
#     )
# )

# fig.add_trace(
#     go.Scatter(
#         name="Otros sabores",
#         mode="markers+lines",
#         x=other["Fecha"],
#         y=other["Diff Vtas Valor"],
#         marker_symbol="star",
#     )
# )

# title = "Crecimiento de Ventas por Cambio de Precios\n<br><i>2022 a 2024</i>"
# fig.update_layout(
#     title=title,
#     title_x=0.5,
#     yaxis_title="Ventas ($) / Cambio de precio",
#     barcornerradius=5,
#     paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
#     plot_bgcolor="rgba(0,0,0,0)",
# )

# fig.add_vrect(
#     x0="2022-01-01",
#     x1="2022-12-31",
#     fillcolor="blue",
#     opacity=0.1,
#     layer="below",
#     line_width=0,
# )

# fig.add_vrect(
#     x0="2023-01-01",
#     x1="2023-12-31",
#     fillcolor="gray",
#     opacity=0.1,
#     layer="below",
#     line_width=0,
# )

# fig.add_vrect(
#     x0="2024-01-01",
#     x1="2024-12-31",
#     fillcolor="blue",
#     opacity=0.1,
#     layer="below",
#     line_width=0,
# )


# fig